# Data Cleaning

The following code will transform the raw data into expected format and report the rows with outliers

In [247]:
import numpy as np
import pandas as pd 
import requests

In [248]:
# file_path for the raw data csv file
file_path = '/Users/fuwang/Documents/Web Scraper/Tickets 2/JapanesePaper/ac_dataset.csv'
df = pd.read_csv(file_path)

In [249]:
df['Report Number'] = df['Report Number'].apply(lambda x: str(x).replace(u'\xa0', ''))
df['Report Number'] = df['Report Number'].replace('',np.nan,regex=True)
df.dropna(axis=0,how='all',inplace=True)

## Extract Time 

In [250]:
# functions that to extract the time from raw data
def extract_time(df):
    for i in range(len(df['Report Number'])):
        if df['Report Number'].iloc[i].startswith('The'):
            df['Report Number'].iloc[i] = df['Report Number'].iloc[i][-8:]
    df.drop(df.tail(3).index,inplace=True) # drop last n rows
    

In [251]:
for i in range(len(df['Report Number'])):
    if df['Report Number'].iloc[i].startswith('The'):
        df['Report Number'].iloc[i] = df['Report Number'].iloc[i][-8:]

df.drop(df.tail(3).index,inplace=True) # drop last n rows
df['Report Number'] = df['Report Number'].str.rstrip(':')

In [252]:
df['Year'] = None
df['File Name'] = None
df = df.astype(str)
for i in range(len(df)):
    if df.iloc[i]['Report Number'][0].isdigit():
        temp = df.iloc[i]['Report Number']
    df['Year'].iloc[i] = temp
    df['File Name'].iloc[i] = df.iloc[i][0]+'.'+df.iloc[i][1]+'.pdf'

In [253]:
index_year = df[df['Report Number'].str[0].str.isdigit()].index
df.drop(index_year,inplace=True)

## Extract Author

In [254]:
df['author'] = df['author'].str.slice(start=3)
df['author'] = df['author'].str.replace(', and',',')
df['author'] = df['author'].str.replace('and',',')
df['author'] = df['author'].apply(lambda x: str(x).replace(u'\xa0', ''))
df['author'] = df['author'].replace(r'\s+', ' ', regex=True)
df.author=df.author.str.split(',')

In [255]:
def extract_authors(df):
    max_num_authors = len(max(df.author,key=len))
    authors = []
    for i in range(max_num_authors):
        authors.append(f'Author {i+1}')
    for i in range(max_num_authors):
        df[f'Author {i+1}'] = None
    for i in range(len(df.author)):
        for j in range(len(df.author.iloc[i])):
            df[f'Author {j+1}'].iloc[i] =  df.author.iloc[i][j].strip()
    for i in range(max_num_authors):
        df[f'Author {i+1}'] = df[f'Author {i+1}'].str.replace(' ',',')
    return authors

In [256]:
authors = extract_authors(df)

## Detect Outlier

In [276]:
def outlier_detect(df):
    outlier_index = []
    outlier_number = []
    df['is_outlier'] = 'no'
    for i in range(df.shape[0]):
        if df['title'].iloc[i] == 'nan' or df.outlier.iloc[i] != 'nan':
            outlier_index.append(i)
            outlier_number.append(df['Report Number'].iloc[i])
            df['is_outlier'].iloc[i] = 'yes'
    return df.iloc[outlier_index]

In [277]:
outlier_df = outlier_detect(df)

In [278]:
# Export Outlier CSV file to expected path
outlier_df.to_csv('/Users/fuwang/Documents/Web Scraper/Tickets 2/JapanesePaper/outlier.csv') 

## Export Metada to CSV file

In [269]:
df_new = df[['title','Year','Report Number'] +authors+
       ['File Name']]
df_new.reset_index(drop=True,inplace=True)

In [270]:
# Export CSV file to the work path you prefered 
df_new.to_csv('/Users/fuwang/Documents/Web Scraper/Tickets 2/JapanesePaper/cleandf.csv') 

## Down PDF files

In [261]:
# convert links 
# allowed_domains = 'business.columbia.edu'
df['downloadable'] = None
for i in range(df.shape[0]):
    if df.link.iloc[i][:6] == '/sites':
        df.link.iloc[i] = 'https://business.columbia.edu' + df.link.iloc[i]
        df['downloadable'].iloc[i] = 'direct link'
    elif df.link.iloc[i][:10] == 'http://hdl' or df['is_outlier'].iloc[i]=='yes':
        df['downloadable'].iloc[i] = 'invalid link'
    else: df['downloadable'].iloc[i] = 'indirect link'

In [262]:
def downloadFile(url, fileName):
    with open(fileName, "wb") as file:
        response = requests.get(url, timeout = 0.5)
        file.write(response.content)

In [ ]:
# This will download all pdf files which has a valid direct downloadable link
for i in range(df.shape[0]):
    if df.is_outlier.iloc[i] != 'yes':
        try:
            downloadFile(df.link.iloc[i],df['File Name'].iloc[i])
        except requests.exceptions.Timeout:
              print ("Timeout occurred")        
    else: continue

In [263]:
# get the files need to be download mannually
def manual_download(df):
    report_number = []
    link = []
    for i in range(df.shape[0]):
        if df['downloadable'].iloc[i] == 'indirect link':
            report_number.append(df['Report Number'].iloc[i])
            link.append(df['link'].iloc[i])
    
    return report_number,link

In [264]:
report, link = manual_download(df)

In [265]:
manual_df = pd.DataFrame({'Report Number':report,'Link':link})

In [266]:
# Export manually CSV file to the work path you prefered 
manual_df.to_csv('/Users/fuwang/Documents/Web Scraper/Tickets 2/JapanesePaper/indirect_link.csv') 